In [50]:
import argparse
from pathlib import Path
import sys

import numpy as np
from numpy.lib.recfunctions import structured_to_unstructured
import rospy
# import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2
from jsk_recognition_msgs.msg import BoundingBox, BoundingBoxArray
from visualization_msgs.msg import Marker,MarkerArray
import ros_numpy

from pyquaternion import Quaternion


import torch
import os
import cv2
import pdb

from mmdet3d.apis import inference_detector, init_model, show_result_meshlab
from mmdet3d.core.bbox.structures.box_3d_mode import Box3DMode, LiDARInstance3DBoxes

from copy import deepcopy

from mmdet3d.datasets.pipelines import Compose
from mmcv.parallel import collate, scatter

In [36]:
config = "mmdetection3d/configs/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class.py"
checkpoint = "mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth"

device = "cuda:0"

data_path = "mmdetection3d/demo/data/kitti/kitti_000008.bin"

model = init_model(config, checkpoint, device=device)


box_type_3d = LiDARInstance3DBoxes
box_mode_3d = Box3DMode.LIDAR


load checkpoint from local path: mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


In [39]:
test_pipeline = deepcopy(model.cfg.data.test.pipeline)

In [40]:
test_pipeline = Compose(test_pipeline)

In [54]:
data = dict(
            pts_filename=data_path,
            box_type_3d=box_type_3d,
            box_mode_3d=box_mode_3d,
            # for ScanNet demo we need axis_align_matrix
            ann_info=dict(axis_align_matrix=np.eye(4)),
            sweeps=[],
            # set timestamp = 0
            timestamp=[0],
            img_fields=[],
            bbox3d_fields=[],
            pts_mask_fields=[],
            pts_seg_fields=[],
            bbox_fields=[],
            mask_fields=[],
            seg_fields=[])

In [55]:
data = test_pipeline(data)

In [57]:
data = collate([data], samples_per_gpu=1)
data['img_metas'] = data['img_metas'][0].data
data['points'] = data['points'][0].data

In [ ]:
with torch.no_grad():
    result = model(return_loss=False, rescale=True, **data)